In [12]:
import torch
import torch.nn.functional as F
from torch import nn

1. 简单的参数管理访问

In [2]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.randn(size = (2, 4))
net(X)

tensor([[0.3012],
        [0.4324]], grad_fn=<AddmmBackward0>)

In [ ]:
## 访问第二个全连接层的参数
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.0525,  0.2487, -0.3216, -0.1045,  0.3281, -0.0535,  0.1625,  0.0748]])), ('bias', tensor([0.2787]))])


2. 访问目标参数

In [ ]:
## 访问第二个全连接层的参数信息
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.2787], requires_grad=True)
tensor([0.2787])


In [7]:
## 由于还没有进行反向传播，所以梯度不存在
net[2].bias.grad == None

True

3. 一次性访问所有参数

In [8]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [10]:
net.state_dict()['2.bias'].data

tensor([0.2787])

4. 从嵌套块收集参数

In [11]:
## 首先定义一个组合块
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

In [13]:
rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [14]:
## 访问第一个主要的块中第二个子块的第一层的bias
rgnet[0][1][0].bias.data

tensor([ 0.0139,  0.0127,  0.4715, -0.2153, -0.1062,  0.3706,  0.2181,  0.0025])

5. 参数初始化

In [20]:
## 初始化权重符合w~N(0, 0.01)分布，bias设为0
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean = 0, std = 0.01)
        nn.init.zeros_(m.bias)
    
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0158,  0.0091, -0.0054,  0.0152]), tensor(0.))

In [22]:
## 将所有参数初始化为常量1
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

6. 自定义初始化

In [ ]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        ## 只保留|w|大于等于5的
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 7.7361, -8.9963,  7.3802,  6.6444],
        [ 0.0000,  6.0657,  8.4043, -9.5945]], grad_fn=<SliceBackward0>)

7. 在多个层中共享参数

In [4]:
## 定义一个稠密层（全连接层），用来引用参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))

net(X)

## 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])

## 改变某一层参数的值，再看看是否相等，确保它们实际上是同一个对象，而不是相同的值
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


8. 延迟初始化

In [5]:
class LazyNet(nn.Module):
    def __init__(self):
        super(LazyNet, self).__init__()
        self.layer1 = None  # 延迟初始化 layer1
        self.layer2 = None  # 延迟初始化 layer2

    def forward(self, x):
        if self.layer1 is None:
            self.layer1 = nn.Linear(10, 20)  # 在第一次访问时才初始化 layer1
        x = self.layer1(x)
        
        if self.layer2 is None:
            self.layer2 = nn.Linear(20, 30)  # 在第一次访问时才初始化 layer2
        x = self.layer2(x)
        
        return x


net = LazyNet()
input_data = torch.randn(5, 10)

output = net(input_data)  # 第一次调用时才会初始化 layer1 和 layer2
print(output.shape)

torch.Size([5, 30])


9. 自定义层（不带参数版）

In [6]:
## 创建一个不含参数的层，用于对传播的数据进行：输入-均值操作
class CenteredLayer(nn.Module):
    ## 继承基本层类并实现前向传播
    def __init__(self):
        super().__init__()
    
    def forward(self, X):
        return X - X.mean()

In [10]:
layer = CenteredLayer()
layer(torch.FloatTensor([1,2,3,4,5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [11]:
## 加到更复杂的模型中
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())
Y = net(torch.rand(4, 8))
Y.mean()

tensor(1.8626e-09, grad_fn=<MeanBackward0>)

10. 带参数的层

In [15]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units, ))
    
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [16]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 1.3471, -0.4014, -0.2832],
        [-1.6407,  1.3450, -0.4389],
        [ 0.8676,  0.6828,  0.4644],
        [-0.5354,  0.2077,  0.5589],
        [-0.2535,  1.2967,  0.5826]], requires_grad=True)

GPU与CPU

In [17]:
X.device

device(type='cpu')